In [ ]:
# 이게 2024.01.14 12:58 기준 최신본.
# TODO: Pattern List에 대한 정보를 json으로 넣는 코드.
# ++ Bezier Curve, Straight가 아닌
# Arc , Qudratic Curve, Cubic Curve에 대해서는 어떻게 다룰지. 

import pandas as pd
import json
import math

# 패널 데이터프레임 생성
panel_dataframes = [process_path_data(drawn_pattern) for drawn_pattern in drawn_pattern_list]
panel_names = list(pattern.panel_order())

# Initialize the top-level JSON structure
garment_data = {
    "FabricName": [],
    "GradingRuleTableList": [],
    "PatternList": [],
    "SymmetricDataList": [],
    "InstanceDataList": [],
    "SeamLinePairGroupList": [],
}

# Process each panel
for panel_name, df in zip(panel_names, panel_dataframes):
    # Initialize JSON structure for the panel
    panel_data = {
        "Name": panel_name,
        "fGrainlineAngle": 0.0,
        "ID": panel_name,
        "strSuperImposeSide": "None",
        "CurrentFabricUUID": "",
        "IsClosed": False,
        "InternalLineList": [],
        "ButtonHeadList": [],
        "ButtonHoleList": [],
        "AnnotationList": [],
        "NotchList": [],
        "IsHalfSymmetric": False,
        "ShapeInfo": {
            "IsSlashed": False,
            "LineList": [],
            "HalfSymmetryPointIDMap": {},
            "HalfSymmetryLineIDMap": {}
        },
        "ArrangementPointDataMap": {
                "PointName": "Body_Front_Center_3",
                "fOffSetX": 0.0,
                "fOffSetY": 0.05,
                "fAngle": 0.0
        }
    }

    # 패널별 좌표 ID 초기화
    coordinate_id_map = {}
    point_counter = 0  # 패널 내 좌표 카운터

    # Generate LineList and PointList for the panel
    for edge_idx, row in df.iterrows():
        line_id = f"{panel_name}_edge_{edge_idx}"
        point_list = []

        # Iterate through columns 2 by 2 for x and y
        for col_x, col_y in zip(row.index[1::2], row.index[2::2]):  # Skip 'type' and go by pairs
            x, y = row[col_x], row[col_y]
            if not (pd.isna(x) or pd.isna(y) or math.isclose(x, 0.0) and math.isclose(y, 0.0)):
                # 좌표에 대해 새로운 ID 생성 (패널 내에서만)
                key = (x, y)
                if key not in coordinate_id_map:
                    coordinate_id_map[key] = f"{panel_name}_point_{point_counter}"
                    point_counter += 1
                point_id = coordinate_id_map[key]

                point_list.append({
                    "ID": point_id,
                    "PointType": row["type"],
                    "Position": {"x": x, "y": y},
                    "GradingRuleID": -1
                })

        # Add Line to LineList
        line = {"ID": line_id, "PointList": point_list}
        panel_data["ShapeInfo"]["LineList"].append(line)

    # Add the panel data to the PatternList
    garment_data["PatternList"].append(panel_data)

# Convert to JSON string with indentation
json_output = json.dumps(garment_data, indent=4)

# Save to a file (optional)
output_file = "multiple_panels_data3.json"
with open(output_file, "w") as file:
    file.write(json_output)

# 결과 출력
print(json_output)

In [ ]:
# 2024.01.14 14:33 기준 2:51 기준
# TODO: Seam Line Pair Group json으로 넣는 코드
# SeamLinePairGroupList 생성
garment_data["SeamLinePairGroupList"] = []

# SeamLinePairGroupList 생성 루프
for idx, value in enumerate(stitch_dict.values()):
    # "Name" 생성
    group_name = f"SeamLineGroup_{idx}"

    # value에서 첫 번째와 두 번째 ShapeID 정보 추출
    first_info = value[0]
    second_info = value[1]

    # 첫 번째 정보
    first_panel = first_info["panel"]
    first_edge = first_info["edge"]
    first_shape_id = f"{first_panel}_edge_{first_edge}"

    # 첫 번째 fStart와 fEnd 추출
    first_fstart_fend = next(
        (item for item in panel_fstart_fend_info[first_panel] if item["edge_id"] == first_edge),
        None
    )
    first_fstart = first_fstart_fend["fstart"]
    first_fend = first_fstart_fend["fend"]

    # 두 번째 정보
    second_panel = second_info["panel"]
    second_edge = second_info["edge"]
    second_shape_id = f"{second_panel}_edge_{second_edge}"

    # 두 번째 fStart와 fEnd 추출
    second_fstart_fend = next(
        (item for item in panel_fstart_fend_info[second_panel] if item["edge_id"] == second_edge),
        None
    )
    second_fstart = second_fstart_fend["fstart"]
    second_fend = second_fstart_fend["fend"]

    # JSON 구조 생성
    seam_line_pair_group = {
        "Name": group_name,
        "bIsTurned": False,
        "PairList": [
            {
                "First": {
                    "ShapeID": first_shape_id,
                    "LengthParam": {
                        "fStart": first_fstart,
                        "fEnd": first_fend
                    },
                    "Direction": True
                },
                "Second": {
                    "ShapeID": second_shape_id,
                    "LengthParam": {
                        "fStart": second_fstart,
                        "fEnd": second_fend
                    },
                    "Direction": False
                }
            }
        ],
        "FoldData": {
            "iAngle": 180,
            "iStrength": 5
        }
    }

    # SeamLinePairGroupList에 추가
    garment_data["SeamLinePairGroupList"].append(seam_line_pair_group)

# JSON 출력
json_output = json.dumps(garment_data, indent=4)

# Save to file (optional)
output_file = "garment_with_seam_lines.json"
with open(output_file, "w") as file:
    file.write(json_output)

# Print the result
print(json_output)

In [ ]:
### 원래꺼 
# 이게 2024.01.14 12:58 + (15:55 기준 최신본.
# TODO: Pattern List + (InstanceDataList)에 대한 정보를 json으로 넣는 코드.
# ++ Bezier Curve, Straight가 아닌
# Arc , Qudratic Curve, Cubic Curve에 대해서는 어떻게 다룰지. 
# InstanceDataList 생성

# InstanceDataList 추가 루프

import pandas as pd
import json
import math

# 패널 데이터프레임 생성
panel_dataframes = [process_path_data(drawn_pattern) for drawn_pattern in drawn_pattern_list]
panel_names = list(pattern.panel_order())

# Initialize the top-level JSON structure
garment_data = {
    "FabricList": [],
    "GradingRuleTableList": [],
    "PatternList": [],
    "SymmetricDataList": [],
    "InstanceDataList": [],
    "SeamLinePairGroupList": [],
}

# Process each panel
for panel_name, df in zip(panel_names, panel_dataframes):
    # Initialize JSON structure for the panel
    panel_data = {
        "Name": panel_name,
        "fGrainlineAngle": 0.0,
        "ID": panel_name,
        "strSuperImposeSide": "None",
        "CurrentFabricUUID": "DWX6ey",
        "IsClosed": False,
        "InternalLineList": [],
        "ButtonHeadList": [],
        "ButtonHoleList": [],
        "AnnotationList": [],
        "NotchList": [],
        "IsHalfSymmetric": False,
        "ShapeInfo": {
            "IsSlashed": False,
            "LineList": [],
            "HalfSymmetryPointIDMap": {},
            "HalfSymmetryLineIDMap": {}
        },
        "ArrangementPointDataMap": {
                "PointName": "Body_Front_Center_3",
                "fOffSetX": 0.0,
                "fOffSetY": 0.05,
                "fAngle": 0.0
        }
    }
    # InstanceDataList에 json에 추가
    instance_data = {
        "OriginPatternID": panel_name,  # 패널 이름을 OriginPatternID로 사용
        "InstancePatternIDArray": []   # 빈 리스트로 추가
    }
    garment_data["InstanceDataList"].append(instance_data)
    # 패널별 좌표 ID 초기화
    coordinate_id_map = {}
    point_counter = 0  # 패널 내 좌표 카운터

    # Generate LineList and PointList for the panel
    for edge_idx, row in df.iterrows():
        line_id = f"{panel_name}_edge_{edge_idx}"
        point_list = []
        if row["type"] == "Bezier Curve":
            row_type = ["Straight", "Bezier Curve", "Bezier Curve", "Straight"]
            idx = 0
            for col_x, col_y in zip(row.index[1::2], row.index[2::2]):  # Skip 'type' and go by pairs
                x, y = row[col_x], row[col_y]
                if not (pd.isna(x) or pd.isna(y)):#TODO: or math.isclose(x, 0.0) and math.isclose(y, 0.0) ):
                    # 좌표에 대해 새로운 ID 생성 (패널 내에서만)
                    key = (x, y)
                    if key not in coordinate_id_map:
                        coordinate_id_map[key] = f"{panel_name}_point_{point_counter}"
                        point_counter += 1
                    point_id = coordinate_id_map[key]
                    
                    point_list.append({
                        "ID": point_id, 
                        "PointType": row_type[idx],
                        "Position": {"x": x, "y": y},
                        "GradingRuleID": -1
                    })
                    idx +=1        
        if row["type"] == "Straight":
            # Iterate through columns 2 by 2 for x and y
            for col_x, col_y in zip(row.index[1::2], row.index[2::2]):  # Skip 'type' and go by pairs
                x, y = row[col_x], row[col_y]
                if not (pd.isna(x) or pd.isna(y)):#TODO: or math.isclose(x, 0.0) and math.isclose(y, 0.0) ):
                    # 좌표에 대해 새로운 ID 생성 (패널 내에서만)
                    key = (x, y)
                    if key not in coordinate_id_map:
                        coordinate_id_map[key] = f"{panel_name}_point_{point_counter}"
                        point_counter += 1
                    point_id = coordinate_id_map[key]
                    
                    point_list.append({
                        "ID": point_id,
                        "PointType": row["type"],
                        "Position": {"x": x, "y": y},
                        "GradingRuleID": -1
                    })

        # Add Line to LineList
        line = {"ID": line_id, "PointList": point_list}
        panel_data["ShapeInfo"]["LineList"].append(line)

    # Add the panel data to the PatternList
    garment_data["PatternList"].append(panel_data)

# Convert to JSON string with indentation
json_output = json.dumps(garment_data, indent=4)

# Save to a file (optional)
output_file = "multiple_panels_data3.json"
with open(output_file, "w") as file:
    file.write(json_output)

# 결과 출력
print(json_output)

In [ ]:
#### 임시
# Generate LineList and PointList for the panel
for edge_idx, row in df.iterrows():
    line_id = f"{panel_name}_edge_{edge_idx}"
    point_list = []

    if row["type"] == "Bezier Curve":
        row_type = ["Straight", "Bezier Curve", "Bezier Curve", "Straight"]
        idx = 0
        for col_x, col_y in zip(row.index[1::2], row.index[2::2]):  # Skip 'type' and go by pairs
            x, y = row[col_x], row[col_y]
            if not (pd.isna(x) or pd.isna(y)):  # 중복 좌표를 무시하지 않음
                # 고유 ID 생성 (중복 좌표도 별도 처리)
                point_id = f"{panel_name}_point_{point_counter}"
                point_counter += 1

                point_list.append({
                    "ID": point_id,
                    "PointType": row_type[idx],
                    "Position": {"x": x, "y": y},
                    "GradingRuleID": -1
                })
                idx += 1

    if row["type"] == "Straight":
        for col_x, col_y in zip(row.index[1::2], row.index[2::2]):  # Skip 'type' and go by pairs
            x, y = row[col_x], row[col_y]
            if not (pd.isna(x) or pd.isna(y)):  # 중복 좌표를 무시하지 않음
                # 고유 ID 생성 (중복 좌표도 별도 처리)
                point_id = f"{panel_name}_point_{point_counter}"
                point_counter += 1

                point_list.append({
                    "ID": point_id,
                    "PointType": row["type"],
                    "Position": {"x": x, "y": y},
                    "GradingRuleID": -1
                })

    # Add Line to LineList
    line = {"ID": line_id, "PointList": point_list}
    panel_data["ShapeInfo"]["LineList"].append(line)